In [1]:
"""
import os, sys
from google.colab import drive
drive.mount('/content/drive')

my_path = '/content/package'
save_path = '/content/drive/MyDrive/Colab Notebooks/package' ## 패키지가 저장될 경로

os.symlink(save_path, my_path)
sys.path.insert(0, my_path)
"""

"\nimport os, sys\nfrom google.colab import drive\ndrive.mount('/content/drive')\n\nmy_path = '/content/package'\nsave_path = '/content/drive/MyDrive/Colab Notebooks/package' ## 패키지가 저장될 경로\n\nos.symlink(save_path, my_path)\nsys.path.insert(0, my_path)\n"

In [ ]:
# GPU check
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

In [2]:
!nvidia-smi

Sat Sep  9 13:31:22 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   66C    P8    12W /  70W |      0MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### 관련 module 설치

In [3]:
# 관련 module 설치

!pip install -q -U bitsandbytes
!pip install -q -U git+https://github.com/huggingface/transformers.git
!pip install -q -U git+https://github.com/huggingface/peft.git
!pip install -q -U git+https://github.com/huggingface/accelerate.git
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 8.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 82.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 63.9 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.2/251.2 kB 1.7 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [4]:
!pip install -U torch transformers tokenizers accelerate safetensors

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 30.6 MB/s eta 0:00:00


### model load
- Polyglot-ko 12.8B 기반 [Full Finetune v1.1] -  8Bit quantized된 모델
- https://huggingface.co/beomi/KoAlpaca-Polyglot-12.8B/tree/8bit
- 8Bit quantized 모델은 colab 무료계정에서 안 됨

In [ ]:
"""
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="beomi/KoAlpaca-Polyglot-12.8B")
"""


"""
# Load model directly
from transformers import AutoTokenizer, AutoModelForCausalLM

tokenizer = AutoTokenizer.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B")
model = AutoModelForCausalLM.from_pretrained("beomi/KoAlpaca-Polyglot-12.8B")
"""

- 4Bit Quantizated model

In [5]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig

model_id = "EleutherAI/polyglot-ko-12.8b"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})

model.eval()

Loading checkpoint shards:   0%|          | 0/28 [00:00<?, ?it/s]

GPTNeoXForCausalLM(
  (gpt_neox): GPTNeoXModel(
    (embed_in): Embedding(30080, 5120)
    (emb_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-39): 40 x GPTNeoXLayer(
        (input_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_layernorm): LayerNorm((5120,), eps=1e-05, elementwise_affine=True)
        (post_attention_dropout): Dropout(p=0.0, inplace=False)
        (post_mlp_dropout): Dropout(p=0.0, inplace=False)
        (attention): GPTNeoXAttention(
          (rotary_emb): GPTNeoXRotaryEmbedding()
          (query_key_value): Linear4bit(in_features=5120, out_features=15360, bias=True)
          (dense): Linear4bit(in_features=5120, out_features=5120, bias=True)
          (attention_dropout): Dropout(p=0.0, inplace=False)
        )
        (mlp): GPTNeoXMLP(
          (dense_h_to_4h): Linear4bit(in_features=5120, out_features=20480, bias=True)
          (dense_4h_to_h): Linear4bit(in_features=20480, out_features=

In [ ]:
!nvidia-smi

Mon Aug 21 08:37:50 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   53C    P0    28W /  70W |   7915MiB / 15360MiB |     11%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Inference

In [6]:
from datasets import load_dataset

data = load_dataset("beomi/KoAlpaca-v1.1a")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [7]:
print(data)


data = data.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

# data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url'],
        num_rows: 21155
    })
})


Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

In [8]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34, 4253,   29, 1832,
          296, 8678,  270,  750,  283,   15, 2175,  270]])

In [ ]:
PROMPT_DICT = {
    "prompt_input": (
        "Below is an instruction that describes a task, paired with an input that provides further context.\n"
        "아래는 작업을 설명하는 명령어와 추가적 맥락을 제공하는 입력이 짝을 이루는 예제입니다.\n\n"
        "Write a response that appropriately completes the request.\n요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Input(입력):\n{input}\n\n### Response(응답):"
    ),
    "prompt_no_input": (
        "Below is an instruction that describes a task.\n"
        "아래는 작업을 설명하는 명령어입니다.\n\n"
        "Write a response that appropriately completes the request.\n명령어에 따른 요청을 적절히 완료하는 응답을 작성하세요.\n\n"
        "### Instruction(명령어):\n{instruction}\n\n### Response(응답):"
    ),
}

In [ ]:
# inferene memo 참고
'''
def gen(prompt, user_input=None, max_new_tokens=128, temperature=0.5):
    if user_input:
        x = PROMPT_DICT['prompt_input'].format(instruction=prompt, input=user_input)
    else:
        x = PROMPT_DICT['prompt_no_input'].format(instruction=prompt)

    input_ids = tokenizer.encode(x, return_tensors="pt").to('cuda:0')
    gen_tokens = model.generate(
        input_ids,
        max_new_tokens=max_new_tokens,
        num_return_sequences=1,
        temperature=temperature,
        no_repeat_ngram_size=6,
        do_sample=True,

        early_stopping=True,
        eos_token_id=2,

    )
    gen_text = tokenizer.decode(gen_tokens[0], skip_special_tokens=True)

    return gen_text.replace(x, '')
'''

In [9]:
'''
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,

        temperature = 0.5,
    )
    print(tokenizer.decode(gened[0]))
'''


def gen(x):
    q = f"### 질문: {x}\n\n### 답변:"
    # print(q)
    gened = model.generate(
        **tokenizer(
            q,
            return_tensors='pt',
            return_token_type_ids=False
        ).to('cuda'),
        max_new_tokens=50,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
        top_k=50,
        top_p =0.95,
        num_beams=2,
    )
    # print(tokenizer.decode(gened[0]))
    # gened[0] : embedding tensor 형식  >> decode로 decoding해서 다시 string으로
    gen_text = tokenizer.decode(gened[0], skip_special_tokens=True)
    # print(gen_text)

    return gen_text


In [ ]:
# Example usage:
prompt = "Python으로 time을 찾는 코드"
generated_text = gen(prompt)
print("answer :", generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: Python으로 time을 찾는 코드

### 답변: Python으로 time을 찾는 코드를 작성하는 방법은 다음과 같습니다.​1. time.time() 함수를 사용하여 time을 찾습니다.​2. time.sleep() 함수를 사용하여 time을 찾습니다.​3. time.wait() 함수를 사용하여 time을 찾습니다.​4. time.time(


In [ ]:
prompt = " ' 성별 : 여자 , 나이 : 25 ,날씨 : 온도 10, 습도 30, 강수확률 100% ,상황 : 데이트 ,상의 색 : 파란 ,하의 색 : 흰색 '에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘"
generated_text = gen(prompt)
print('answer :', generated_text)
#print({generated_text['output']})

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: '성별 : 여자, 나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 '에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘

### 답변: ####################################


In [ ]:
prompt = input()
generated_text = gen(prompt)
print("answer :", generated_text)

" 성별 : 여자 ,나이 : 25 ,날씨 : 온도 10, 습도 30, 강수확률 100% ,상황 : 데이트 ,상의 색 : 파란 ,하의 색 : 흰색 " 에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘


/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:399: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: " 성별 : 여자,나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 " 에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘

### 답변: 요즘처럼 갑자기 더워지고 또 쌀쌀해지는 환절기 날씨에는 가디건을 걸쳐주시면 좋으실꺼에요. 날씨도 쌀쌀하지 않게 입으실수도있고 또 가디건 하나만 걸치시기에도 좋구요
### 질문: " 성별 : 여자,나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 " 에 대한 한 가지의 옷 추천을 키워드로만 한 줄로 말해줘

### 답변: 요즘처럼 갑자기 더워지고 또 쌀쌀해지는 환절기 날씨에는 가디건을 걸쳐주시면 좋으실꺼에요. 날씨도 쌀쌀하지 않게 입으실수도있고 또 가디건 하나만 걸치시기에도 좋구요
answer : None


In [ ]:
"""
import locale
locale.getdefaultlocale()
locale.getpreferredencoding = lambda: "UTF-8"
"""

'\nimport locale\nlocale.getdefaultlocale()\n'

In [ ]:
!nvidia-smi

Fri Sep  8 07:50:09 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    31W /  70W |   9593MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

### Fine-tuning : PERT

In [10]:
# fine-tuning dataset
import json

filepath = '/content/total_extracted_data.ddata.json'

fine_tuning_dataset = load_dataset('json', data_files=filepath)

'''
with open(filepath) as json_file:
  fine_tuning_dataset = json.load(json_file)
'''

print(fine_tuning_dataset)
print(data)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output'],
        num_rows: 28065
    })
})
DatasetDict({
    train: Dataset({
        features: ['instruction', 'output', 'url', 'text'],
        num_rows: 21155
    })
})


In [11]:
fine_tuning_dataset = fine_tuning_dataset.map(
    lambda x: {'text': f"### 질문: {x['instruction']}\n\n### 답변: {x['output']}<|endoftext|>" }
)

print(fine_tuning_dataset)

Map:   0%|          | 0/28065 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input', 'instruction', 'output', 'text'],
        num_rows: 28065
    })
})


In [12]:
# data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)

fine_tuning_dataset = fine_tuning_dataset.map(lambda samples: tokenizer(samples["text"]), batched=True)

Map:   0%|          | 0/28065 [00:00<?, ? examples/s]

In [13]:
from peft import prepare_model_for_kbit_training

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

In [14]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [15]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["query_key_value"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 6553600 || all params: 6608701440 || trainable%: 0.09916622894073424


In [ ]:
!nvidia-smi

Fri Sep  8 08:25:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.105.17   Driver Version: 525.105.17   CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P0    31W /  70W |   8757MiB / 15360MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [16]:
import transformers

# needed for gpt-neo-x tokenizer
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    # train_dataset=data["train"],
    train_dataset=fine_tuning_dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

You're using a PreTrainedTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,3.465400
20,2.493100
30,2.165200
40,1.862700
50,1.904400
60,1.836700
70,1.777000
80,1.856600
90,1.781900
100,1.676500


TrainOutput(global_step=500, training_loss=1.6809270725250245, metrics={'train_runtime': 1464.8258, 'train_samples_per_second': 0.683, 'train_steps_per_second': 0.341, 'total_flos': 2339412691722240.0, 'train_loss': 1.6809270725250245, 'epoch': 0.04})

In [ ]:
print(data['train'][11883])

{'instruction': '중국에서 출산 후에는 왜 물을 마시지 않고 술을 먹이는 것일까요?', 'output': '중국에서 여성의 출산 후 한 달 동안에는 물마시기를 금하며, 식사에도 물을 사용하지 않습니다. 대신에, 산부는 술을 끓여 알콜성분을 줄여서 마시거나 요리에 사용하여 몸의 필요한 수분을 보충합니다. 그 이유는 중국의학에서 출산 후 한 달 동안은 출산 때문에 이완된 신체의 조직을 본래대로 수축시키는 중요한 시기이기 때문입니다. 이 시기에 수축이 잘되지 않으면 체형의 붕괴나 내장하수 등의 후유증이 남으므로 수축이 잘되지 않는 것을 방지하기 위해서 술을 먹이는 것입니다. 따라서 산부는 매일 술을 마시는 기분으로 지낸다고 합니다.\n 추가설명: 중국의 여성들은 출산 후 체형의 붕괴를 막기 위하여 출산 후 한 달 간 많은 수분을 섭취하거나 신체의 냉기 때문에 건강이 나빠지는 경우가 있습니다. 그렇기 때문에 출산 후 한 달 동안 수분을 흡수하기 위해 적당한 도수의 술을 마시거나 요리에 사용하고, 물이나 얼음을 마시거나 먹으면 체형의 붕괴를 방지하기 어렵습니다.', 'url': 'https://kin.naver.com/qna/detail.naver?d1id=6&dirId=61302&docId=47067375', 'text': '### 질문: 중국에서 출산 후에는 왜 물을 마시지 않고 술을 먹이는 것일까요?\n\n### 답변: 중국에서 여성의 출산 후 한 달 동안에는 물마시기를 금하며, 식사에도 물을 사용하지 않습니다. 대신에, 산부는 술을 끓여 알콜성분을 줄여서 마시거나 요리에 사용하여 몸의 필요한 수분을 보충합니다. 그 이유는 중국의학에서 출산 후 한 달 동안은 출산 때문에 이완된 신체의 조직을 본래대로 수축시키는 중요한 시기이기 때문입니다. 이 시기에 수축이 잘되지 않으면 체형의 붕괴나 내장하수 등의 후유증이 남으므로 수축이 잘되지 않는 것을 방지하기 위해서 술을 먹이는 것입니다. 따라서 산부는 매일 술을 마시는 기분으로 지낸다고 합니다.\n 추가설명:

In [17]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [ ]:
model.generate(**tokenizer("### 질문: 오늘 날씨는?", return_tensors='pt', return_token_type_ids=False))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1260: UserWarning: Using the model-agnostic default `max_length` (=20) to control thegeneration length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1535: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(


tensor([[   6,    6,    6, 2438,   29, 1832, 4770,  272,   34, 3913,   66,  284,
          361,  380, 7015, 6525, 6644,  745,   66,  283]])

In [ ]:
'''
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:",
            return_tensors='pt',
            return_token_type_ids=False
        ),
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))
'''

In [18]:
prompt = " ' 성별 : 여자 , 나이 : 25 ,날씨 : 온도 10, 습도 30, 강수확률 100% ,상황 : 데이트 ,상의 색 : 파란 ,하의 색 : 흰색 '에 대해 옷 추천해줘"
generated_text = gen(prompt)
print('answer :', generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: '성별 : 여자, 나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 '에 대해 옷 추천해줘

### 답변: 화이트 색상의 원피스는 어떠신가요? 이 원피스는 허리선이 높게 있어 다리가 길어보이는 효과가 있습니다. 어떠신가요? 마음에 드시나요? ^^ 원피스와 어울릴만한 신발도


In [19]:
prompt = input()
generated_text = gen(prompt)
print("answer :", generated_text)

성별 : 남자 나이 : 26 날씨 : 온도 29, 습도 67, 강수확률 2% 상황 : 여행 상의 색 : 파랑 하의 색 : 흰색


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: 성별 : 남자 나이 : 26 날씨 : 온도 29, 습도 67, 강수확률 2% 상황 : 여행 상의 색 : 파랑 하의 색 : 흰색

### 답변: 캐주얼하면서 활동적인 느낌을 주는 코디입니다. 어떠신가요? 마음에 드시나요? ^^  마음에 드시는 코디가 있으신가요? 이 외에도 다양한 코디가 준비되어 있으니


In [20]:
# needed for gpt-neo-x tokenizer
data = data.map(lambda samples: tokenizer(samples["text"]), batched=True)
tokenizer.pad_token = tokenizer.eos_token

trainer = transformers.Trainer(
    model=model,
    train_dataset=data["train"],
    # train_dataset=fine_tuning_dataset['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=1,
        # warmup_steps=200,
        max_steps=500, ## 초소형만 학습: 10 step = 20개 샘플만 학습.
        learning_rate=2e-4,
        fp16=True,
        logging_steps=10,
        output_dir="outputs",
        optim="paged_adamw_8bit"
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

Map:   0%|          | 0/21155 [00:00<?, ? examples/s]

Step,Training Loss
10,2.064500
20,1.990300
30,1.903100
40,1.994200
50,1.833700
60,1.955200
70,1.826900
80,1.870100
90,1.805100
100,1.917700


TrainOutput(global_step=500, training_loss=1.8473624801635742, metrics={'train_runtime': 2656.4095, 'train_samples_per_second': 0.376, 'train_steps_per_second': 0.188, 'total_flos': 1.091963455463424e+16, 'train_loss': 1.8473624801635742, 'epoch': 0.05})

In [21]:
prompt = "안녕하세요. 25살 여자인데요, 온도 10, 습도 30, 강수확률 30%로, 데이트에 입고 갈 옷 추천해주세요."
generated_text = gen(prompt)
print('answer :', generated_text)

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1417: UserWarning: You have modified the pretrained model configuration to control generation. This is a deprecated strategy to control generation and will be removed soon, in a future version. Please use a generation configuration file (see https://huggingface.co/docs/transformers/main_classes/text_generation )
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
/usr/local/lib/python3.10/dist-packages/torch/utils/checkpoint.py:31: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn("None of the inputs have requires_grad=True. Gradients will be None")


answer : ### 질문: 안녕하세요. 25살 여자인데요, 온도 10, 습도 30, 강수확률 30%로, 데이트에 입고 갈 옷 추천해주세요.

### 답변: 위 조건에 맞는 코디는 다음과 같습니다. 

1. 상의: 얇은 긴팔 셔츠, 얇은 가디건, 얇은 니트, 반팔 티셔츠, 반팔 셔츠, 반팔 남방, 얇은 긴


In [23]:
prompt = " ' 성별 : 여자 , 나이 : 25 ,날씨 : 온도 10, 습도 30, 강수확률 100% ,상황 : 데이트 ,상의 색 : 파란 ,하의 색 : 흰색 '에 대해 옷 추천해줘"
generated_text = gen(prompt)
print('answer :', generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: '성별 : 여자, 나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색 '에 대해 옷 추천해줘

### 답변:  위의 조건을 만족하는 옷은 다음과 같습니다. 1. 블라우스와 치마로 구성된 코디로, 상의는 흰색 블라우스, 하의는 치마가 좋습니다. 2. 치마와 블라우스가 포함된 코디로, 상의


In [24]:
prompt = "'성별 : 여자 , 나이 : 25 ,날씨 : 온도 10, 습도 30, 강수확률 100% ,상황 : 데이트 ,상의 색 : 파란 ,하의 색 : 흰색' 상황에 대해 한줄로 옷 추천해줘"
generated_text = gen(prompt)
print('answer :', generated_text)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


answer : ### 질문: '성별 : 여자, 나이 : 25,날씨 : 온도 10, 습도 30, 강수확률 100%,상황 : 데이트,상의 색 : 파란,하의 색 : 흰색' 상황에 대해 한줄로 옷 추천해줘

### 답변: 이와 같은 상황에서는 여성스럽고 우아한 느낌을 주는 원피스를 추천드립니다. 원피스는 여성스럽고 우아한 느낌을 주는 대표적인 아이템으로, 다양한 색상과 디자인이 있어 선택의 폭이


In [30]:
# print(type(generated_text))
answer_data = generated_text

start_idx = int(answer_data.find('### 답변:')) + 8
# print(start_idx)
answer_data = answer_data[start_idx:]
print(answer_data)

이와 같은 상황에서는 여성스럽고 우아한 느낌을 주는 원피스를 추천드립니다. 원피스는 여성스럽고 우아한 느낌을 주는 대표적인 아이템으로, 다양한 색상과 디자인이 있어 선택의 폭이


In [31]:
# konlpy 설치
%%bash
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip3 install JPype1
pip3 install konlpy

Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [1,059 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-updates/restricted amd64 Packages [1,079 kB]
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [962 kB]
Get:13 http://archive.ubuntu.com/ubu

E: Package 'python-dev' has no installation candidate


In [32]:
# 환경변수 변경 - Konlpy
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [41]:
"""
# mecab 설치
%%bash
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
"""

'\n# mecab 설치\n%%bash\nbash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)\n'

## keyword 추출
- KoAlpaca로 나온 답변의 의류 keyword 추출
- text preprocessing : 한글 제외 문자 제거 -> 토큰화 -> 불용어 제거


In [59]:
# import
import re
import pandas as pd
import numpy as np
import konlpy.tag
from konlpy.tag import Mecab, Okt
from setuptools import sic           # tokenizer

In [67]:
class Noun_Extractor:
  def __init__(self):
    '''
    self.answer_data_example = ["현재 날씨에서 추천하는 의상은 하늘색 10부 팬츠와 흰색 상의입니다.",
                                '여자에게 추천하는 데이트룩은 "사랑스러운 원피스"입니다.',
                                "오늘 비오는 날에 어울리는 옷차림은 차분한 컬러의 원피스일 것 같습니다.",
                                "파란색 비옷",
                                "하늘하늘한 원피스를 추천드려요!",
                                "'핑크'와 '하얀색'이라는 키워드로 추천하는 옷은 핑크색 블라우스와 하얀색 치마입니다.",
                                "파란색 반팔 티에 흰색 5부 팬츠를 추천합니다.",
                                "상황에 어울리는 깔끔한 '청순한' 파란색 원피스를 추천해드릴게요!"]
    '''
    self.designed_stop_words = ["날씨", "현재", "비", "날", "햇볓", "맑음", '하늘', '색부', '것', '색', '핑크',
                                "하늘색", "핑크색", "하얀색", "파란색", "빨간색", '검정색', "노란색", "주황색", '초록색', "보라색", '흰색',
                                '청순', '사랑', '차분', '스러운', '컬러', '어울리', '티', '부',
                                '여성', '느낌', '대표', '아이템', '색상', '디자인', '선택', '폭',
                                "상황", "추천", "의상", "옷", "키워드",  "여자", "남자", "옷차림", '상의', '하의', '룩', '데이트']
    self.one_stop_words = ['은','는','이','가','하','아','것','들','의','있','되','수', '고', '에서',
                        '보','주','등','한','과','랑', '을', '를', '와', '도', '어요', '다']                   # 지정 불용어
    self.candidate_stop_words = self.Candidate_Stopwords_list_Load()                                    # 한국어불용어100사전
    self.stop_words = list(set(self.one_stop_words + self.candidate_stop_words + self.designed_stop_words))       # 가게 이름까지 합친 불용어

    # self.text_preprocessing()


  def Candidate_Stopwords_list_Load(self):
    '''
    Candidate_Stopwords_list_Load() : 한국어불용어100.text 파일을 열어서 불용어 단어 반환
        input parameter : None
        output : (list)불용어
    '''
    file_name = '/content/한국어불용어100.txt'
    korean_stopwords100 = pd.read_csv(file_name, sep = '\t', header = None, names=['word', 'morph', 'rate'])
    return korean_stopwords100['word'].tolist()


  def cleaning(self, answer):
    '''
    cleaning() : 데이터 정제
                토큰화(Tokenization)
                불용어(Stopword)
                정규 표현식 (Regular Expression)
                정수 인코딩(Integer Encoding)
        input parameter : (str)answer
        output : (str)정제된 정보
    '''
    print("전처리할 answer text :", answer)
    self.tag = Okt()
    if type(answer) == str:                 # main_text가 빈 리스트가 아닌 경우
        # 영어 및 한글을 제외한 문자 모두 제거
        # answer_text = re.sub('[^A-Za-z가-힣]','',answer)
        answer_text = re.sub('[^가-힣0-9]','',answer)
        #print('answer_text :', answer_text)

        # 영어는 소문자로 convert
        # cleaned_lowercase = answer_text.lower()

        # 형태소 토큰화 - 명사 추출
        # answer_words = self.tag.morphs(cleaned_lowercase)
        # answer_words = self.tag.morphs(answer_text)
        answer_words = self.tag.nouns(answer_text)
        print('answer_words :', answer_words)

        # 불용어 제거
        cleaned_words = [token for token in answer_words if not token in self.stop_words]
        print('cleaned_words :', cleaned_words)
    else:
        cleaned_words = []

    cleaned_words = list(set(cleaned_words))

    # return " ".join(cleaned_words)    # 하나의 문장으로
    return cleaned_words                # list로


  def text_preprocessing(self, answer_data):
    # self.tag = Mecab()
    self.tag = Okt()

    '''
    answer_data = self.answer_data_example
    noun_list = self.cleaning(answer_data)
    #print(noun_list)
    '''

    print("답변 data :", answer_data)

    data=pd.DataFrame()
    # data['answer'] = self.answer_data_example
    data['answer'] = answer_data
    data['keyword'] = data['answer'].apply(self.cleaning)

    print(data)

    return data

In [49]:
print(answer_data)

이와 같은 상황에서는 여성스럽고 우아한 느낌을 주는 원피스를 추천드립니다. 원피스는 여성스럽고 우아한 느낌을 주는 대표적인 아이템으로, 다양한 색상과 디자인이 있어 선택의 폭이


In [68]:
'''
answer_data = generated_text
start_idx = int(answer_data.find('### 답변:')) + 8
answer_data = answer_data[start_idx:]
'''

noun_extractor = Noun_Extractor()
noun_keyword = noun_extractor.cleaning(answer_data)
print("의류추천 Keyword :", noun_keyword)


전처리할 answer text : 이와 같은 상황에서는 여성스럽고 우아한 느낌을 주는 원피스를 추천드립니다. 원피스는 여성스럽고 우아한 느낌을 주는 대표적인 아이템으로, 다양한 색상과 디자인이 있어 선택의 폭이
answer_words : ['이', '상황', '여성', '느낌', '원피스', '추천', '원피스', '여성', '느낌', '대표', '아이템', '색상', '디자인', '선택', '폭']
cleaned_words : ['원피스', '원피스']
의류추천 Keyword : ['원피스']


In [69]:
model.save_pretrained("./KoAlpaca_4bit_save")

In [71]:
# model_filename = './KoAlpaca_4bit_save' + '.hdf5'
# model.save(model_filename)